In [11]:
#brainfuck parser

#functions:
#print: todo
#KNOWN ISSUE. TEMP MIGHT BE ALLOCATED TWICE FOR FOLLOWING OPS MAYBE
pseudoCode = """

input(a);
input(unused);
input(b);
b -= 48;
c = a - b;
d = a + b;
print(c);
print(d);

"""
pseudoCode = pseudoCode.replace('\n',"")
dirtyLines = pseudoCode.split(";")
lines = []
for i in dirtyLines:
    if i != "" and ("=" in i or "(" in i or "{" in i or "}" in i):
        lines.append(i)


dataPointer = 0
#returns string of chars that will move the data pointer to the desired position
def fetchData(index):
    global dataPointer
    assert(index >= 0)
    move = index - dataPointer
    dataPointer = index
    if move < 0:
        return "<" * (move * -1)
    else:
        return ">" * move
#returns string of chars that will increment the current data pointer n times
#if n negative, decrement
def increment(n):
    if n < 0:
        return "-" * (n * -1)
    else:
        return "+" * n
    
def move(f, t):
    return fetchData(f)+"[" + fetchData(t) + "+" + fetchData(f)+"-]"
    
brainfuckCommands = []
brainfuck = ""

#alternating between temp and variables
variables = {}
# are on temp cells. Temp allocator needs to take
constants = {} 
nextVariableLocation = 1

#add command to the final brainfuck output
def append(command):
    brainfuckCommands.append(command)
    
#prepares temporary values for use, clears them if needed
def allocTemps(index, amount):
    result = []
    index -= index%2#align properly
    for i in range(amount):
        a = 0
        if i - 1> index:
            a = i * 2
        elif i % 2 == 0:
            a = index + i + 2
        else:
            a = index - i + 1
        result.append(a)
    return result
    
    
def getVariableLocation():
    global nextVariableLocation
    location = nextVariableLocation
    #first and every second one is temp
    nextVariableLocation += 2
    return location

#sets to 0
def clear(index):
    return fetchData(index) + "[-]"

def addTo(x, t, y):
    print("increment:", x, t, y)
    return fetchData(x) + "["+ fetchData(y) + "+" + fetchData(t) + "+" + fetchData(x) + "-]" + move(t, x)
def substractFrom(x, t, y):
    print("decrement:", x, t, y)
    return fetchData(x) + "["+ fetchData(y) + "-" + fetchData(t) + "+" + fetchData(x) + "-]" + move(t, x)
def add(x0, x1, t, y):
    print("add:", x0, x1, t, y)
    return clear(y) + addTo(x0, t, y) + addTo(x1, t, y)
def substract(x0, x1, t, y):
    print("substract:", x0, x1, t, y)
    return clear(y) + addTo(x0, t, y) + substractFrom(x1, t, y)

#returns index, where the value is, or will be
#temp for safety if needed
def getIndex(expression, temp):
    if isnumber(expression):
        append(fetchData(temp) + increment(int(expression)))
        return temp
    elif expression in variables:
        return variables[expression][0]
    else:
        print("creating a new variable", expression)
        variables[expression] = (getVariableLocation(),0)
        return variables[expression][0]
    
for i in lines:
    i = i.replace(" ", "")
    if i.startswith("#"):
        continue
    #find variable
    equals = i.find("=")
    l=0
    decMult = 1
    incOperator = i.find('+=')
    if incOperator == -1:
        incOperator = i.find('-=')
        decMult = -1
    if incOperator != -1:
        equals -= 1
        l =1
    if equals != -1:
        var = i[0:equals]
        expression = i[equals + 1 + l:]
        if(incOperator == -1):
            try:
                #here static single value assigments
                evaluated = eval(expression)
                variables[var] = (getVariableLocation(), evaluated) #index, value
                #generate code for it
                append(fetchData(variables[var][0]))
                append(increment(variables[var][1]))
                continue
            except:
                pass
        #anyways, allocate at least space for yet unknown variables
        if var not in variables:
            variables[var] = (getVariableLocation(), 0)
    else:
        expression = i
        var = None
    #could not be statically evaluated
    #so far only one operation in line is supported
    #allocate pool of temps 
    if var == None:
        resultPosition = dataPointer
    else:
        resultPosition = variables[var][0]
    temps = allocTemps(resultPosition, 2)
    
    #INC DEC
    if incOperator != -1:
        if decMult == 1:
            append(addTo(getIndex(expression, temps[0]), temps[1], resultPosition))
        else:
            append(substractFrom(getIndex(expression, temps[0]), temps[1], resultPosition))
        continue
    #ADD
    addOperator = expression.find('+')
    if addOperator != -1:
        #if it required two temps - all numbers, it would already be evaluated previously
        append(add(getIndex(expression[0:addOperator], temps[0]), getIndex(expression[addOperator + 1:], temps[0]), temps[1], variables[var][0]))
        continue
    #SUBSTRACT
    subOperator = expression.find('-')
    if subOperator != -1:
        #if it required two temps - all numbers, it would already be evaluated previously
        append(substract(getIndex(expression[0:subOperator], temps[0]), getIndex(expression[subOperator + 1:], temps[0]), temps[1], variables[var][0]))
        continue
    #PRINT
    if expression.find("print") != -1:
        append(fetchData(getIndex(expression[expression.find('(') + 1 : expression.find(')')], temps[0])))
        append(".")
        continue
    #INPUT
    if expression.find("input") != -1:
        append(fetchData(getIndex(expression[expression.find('(') + 1 : expression.find(')')], temps[0])))
        append(",")
        continue
brainfuck = "".join(brainfuckCommands)
print(brainfuck)

creating a new variable a
creating a new variable unused
creating a new variable b
decrement: 6 4 5
substract: 1 5 6 7
increment: 1 6 7
decrement: 5 6 7
add: 1 5 8 9
increment: 1 8 9
increment: 5 8 9
>,>>,>>,>++++++++++++++++++++++++++++++++++++++++++++++++[<-<+>>-]<<[>>+<<-]>>>[-]<<<<<<[>>>>>>+<+<<<<<-]>>>>>[<<<<<+>>>>>-]<[>>-<+<-]>[<+>-]>>>[-]<<<<<<<<[>>>>>>>>+<+<<<<<<<-]>>>>>>>[<<<<<<<+>>>>>>>-]<<<[>>>>+<+<<<-]>>>[<<<+>>>-]<.>>.


In [23]:
#brainfuck parser v2
pseudoCode = """
pre = 49;
c = pre + pre;
print(c);
d = c + 16;
print(d);
c -= 1;
print(c);
d -=9;
print(d);
d += 5;
print(d);
pre -= 17;
print(pre);
c += 5;
print(c);
e = d + 7;
print(e);
c -= 3;
print(c);
d -= 3;
print(d);
pre +=1;
print(pre);
"""

dataPointer = 0
#all other are guaranteed to be 0
#@ is for temp variables
variables = {}
#ready for recycling
#currently not used
unusedCells = {}

brainfuckCommands = []

pseudoCode = pseudoCode.replace('\n',"")
dirtyLines = pseudoCode.split(";")
lines = []
for i in dirtyLines:
    if (i != "" and ("=" in i or "(" in i or "{" in i or "}" in i) and not "#" in i):
        i = i.replace(" ", "")
        lines.append(i)
        
#around dataPointer
def allocateVariable(var):
    #chance to fill previous
    cell = dataPointer - 1
    while True:
        if cell < 0:
            cell = 0
            continue
        free = True
        for i in variables:
            if variables[i] == cell:
                free = False
                break
        if free:
            realloc = False
            if cell in unusedCells:
                del(unusedCells[cell])
                
            if var in variables:
                if variables[var] != cell:
                    unusedCells[variables[var]] = None
                    realloc = True
            variables[var] = cell
            print("reallocating" if realloc else "allocating new" + " variable:", var, " at location:", cell)
            break
        cell += 1
        
#for temps
def freeVariable(var):
    del variables[var]
    print("freed variable:", var)

def parseLine(string):
    found = string.find("=")
    specialOperators = ['+', '-']
    if found != -1:
        if string[found - 1] in specialOperators:
            return (string[0:found - 1], string[found - 1:])
        else:
            return (string[0:found], string[found+1:])
    else:
        return("", string)

def isnumber(x):
    try:
        int(x)
        return True
    except ValueError:
        return False
    
def fetchData(index):
    global dataPointer
    assert(index >= 0)
    move = index - dataPointer
    dataPointer = index
    if move < 0:
        brainfuckCommands.append("<" * (move * -1))
    else:
        brainfuckCommands.append(">" * move)
        
def changeData(n):
    if n < 0:
        brainfuckCommands.append("-" * (n * -1))
    else:
        brainfuckCommands.append("+" * n)
    
###BEGIN OPERATORS###
def increment(context):
    if context[1].startswith("+="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '+' increment
        if isnumber(context[1]):
            assignConstant(context)
        else:
            xIndex = variables[context[1]]
            allocateVariable(context[1])
            
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]])
            brainfuckCommands.append("+")
            fetchData(variables[context[1]])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False
            
def decrement(context):
    if context[1].startswith("-="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '-' decrement
        if isnumber(context[1]):
            if not context[1].startswith('-'):
                context = (context[0],"-" + context[1])
            else:
                context = (context[0],context[1][1:])
            assignConstant(context)
        else:
            xIndex = variables[context[1]]
            allocateVariable(context[1])
            
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]])
            brainfuckCommands.append("-")
            fetchData(variables[context[1]])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False

def add(context):
    plusIndex = context[1].find("+")
    if plusIndex != -1:
        x0 = context[1][:plusIndex]
        x1 = context[1][plusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        increment((context[0], "+=" + x1, context[2]))
        return True
    return False
        
def substract(context):
    minusIndex = context[1].find("-")
    if minusIndex != -1:
        x0 = context[1][:minusIndex]
        x1 = context[1][minusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        decrement((context[0], "-=" + x1, context[2]))
        return True
    return False

def getInput(context):
    if context[1].find("input(") != -1:
        expression = context[1][6:-1]
        allocateVariable(expression)
        fetchData(variables[expression])
        brainfuckCommands.append(',')
        return True
    return False

def printOutput(context):
    if context[1].find("print(") != -1:
        expression = context[1][6:-1]
        if isnumber(expression):
            allocateVariable("@")
            fetchData(variables["@"])
            changeData(int(expression))
            freeVariable("@")
        else:
            fetchData(variables[expression])
        brainfuckCommands.append('.')
        return True
    return False

#always last!
def assignConstant(context):
    if isnumber(context[1]):
        fetchData(variables[context[0]])
        changeData(int(context[1]))
    else:
        increment((context[0], "+=" + context[1], context[2]))
    return True
###END OPERATORS###
    
for i in lines:
    parsed = parseLine(i)
    if parsed[0] != "" and parsed[0] not in variables:
        allocateVariable(parsed[0])
        parsed = (parsed[0], parsed[1], False)
    else:
        parsed = (parsed[0], parsed[1], True)
                  
    #check for possible operations
    if increment(parsed):
        continue
    if decrement(parsed):
        continue
    if add(parsed):
        continue
    if substract(parsed):
        continue
    if getInput(parsed):
        continue
    if printOutput(parsed):
        continue  
    if assignConstant(parsed):
        continue
    
brainfuck = "".join(brainfuckCommands)
print("\n\n######################OUTPUT#########################")
print(brainfuck)

allocating new variable: pre  at location: 0
allocating new variable: c  at location: 1
reallocating pre  at location: 2
reallocating pre  at location: 0
allocating new variable: d  at location: 2
reallocating c  at location: 3
allocating new variable: e  at location: 4
reallocating d  at location: 5


######################OUTPUT#########################
+++++++++++++++++++++++++++++++++++++++++++++++++[>+>+<<-]>>[<+<+>>-]<.[>+>+<<-]>++++++++++++++++.>-.<---------.+++++.<<-----------------.>>>+++++.<[>>+>+<<<-]>>+++++++.<---.>>---.<<<<<+.
